In [68]:
import sys
sys.path.append("/vol/miltank/users/kaiserj/Clipping_vs_Sampling/")
import shutil
import json
from pathlib import Path
from typing import List, Dict, Any
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import warnings
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
from itertools import combinations
import statsmodels.formula.api as smf
from collections import defaultdict
from joblib import Parallel, delayed
from opacus_new import PrivacyEngine
from opacus_new.accountants import RDPAccountant
from opacus_new.validators.module_validator import ModuleValidator

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models

import numpy as np

from types import SimpleNamespace
from tqdm import tqdm
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="Secure RNG turned off.*")
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [69]:
## Define a custom dataset class

class customSmallDS(Dataset):
    def __init__(self, data, target):
        self.tensors = [data, target]

class CustomDataset(Dataset):
    def __init__(self, num_samples=10_000, height=32, width=32, num_features=3, transform=None, target_transform=None):
        self.data = (255 * np.random.rand(num_samples, height, width, num_features)).astype(np.uint8)
        self.targets = torch.randint(0, 10, (num_samples,))
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = customSmallDS(self.data, self.targets)
        self.indices = list(range(num_samples))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]

        # Apply transformations if any
        if self.transform:
            sample = self.transform(sample)

        if self.target_transform:
            target = self.target_transform(target)

        return sample, target
    
@classmethod
def from_dataset(cls, dataset):
    # If we got a Subset, unwrap it
    if isinstance(dataset, torch.utils.data.Subset):
        data = dataset.dataset.data[dataset.indices]
        targets = dataset.dataset.targets[dataset.indices]
        transform = getattr(dataset.dataset, "transform", None)
        target_transform = getattr(dataset.dataset, "target_transform", None)
    else:
        data = dataset.data
        targets = dataset.targets
        transform = getattr(dataset, "transform", None)
        target_transform = getattr(dataset, "target_transform", None)

    return cls(
        data=data,
        targets=targets,
        transform=transform,
        target_transform=target_transform,
    )

## Define a custom model class

class DummyResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.model = models.resnet18(num_classes=num_classes)

    def forward(self, x):
        return self.model(x)



def make_private(model, train_loader, pp_budgets, args):
    modulevalidator = ModuleValidator()
    model = modulevalidator.fix_and_validate(model)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    privacy_engine = PrivacyEngine(accountant=args.accountant,
                                   individualize=args.individualize,
                                   weights=args.weights,
                                   pp_budgets=pp_budgets)
    if args.adapt_weights_to_budgets:
        private_model, private_optimizer, private_loader = privacy_engine \
            .make_private_with_epsilon(module=model,
                                       optimizer=optimizer,
                                       data_loader=train_loader,
                                       target_epsilon=min(pp_budgets),
                                       target_delta=args.target_delta,
                                       epochs=args.epochs,
                                       max_grad_norm=args.max_grad_norm,
                                       optimal=True,
                                       max_alpha=10_000)
                                    #    numeric=True)
    else:
        private_model, private_optimizer, private_loader = privacy_engine \
            .make_private(module=model,
                          optimizer=optimizer,
                          data_loader=train_loader,
                          noise_multiplier=args.noise_multiplier,
                          max_grad_norm=args.max_grad_norm)

    if args.individualize == 'clipping':
        print("Maxgradnorm:", privacy_engine.weights)
        print("budgets:", list(np.unique(np.array(pp_budgets), return_counts=True)))
        print("noise_multiplier:", [private_optimizer.noise_multiplier * args.max_grad_norm/priv for priv in privacy_engine.weights])
        print("weights:", privacy_engine.weights)
        print()
        return{
            "budgets": list(np.unique(np.array(pp_budgets))),
            "max_grad_norms": privacy_engine.weights,
            "sample_rate": [1 / len(private_loader)] * len(privacy_engine.weights),
            "noise_multiplier": [private_optimizer.noise_multiplier for priv in privacy_engine.weights] 
        }
    elif args.individualize == 'sampling':
        return{
            "budgets": list(np.unique(np.array(pp_budgets))),
            "max_grad_norms": [args.max_grad_norm] * len(privacy_engine.weights),
            "sample_rate":privacy_engine.weights,
            "noise_multiplier":[private_optimizer.noise_multiplier] * len(privacy_engine.weights)
        }
    else:
        return

In [70]:
def generate_latex_label(portions: str, bold_index: int) -> str:
    """
    rf"$\mathbf{{{math.ceil(p[0]*100)}\%}}$/{math.ceil(p[1]*100)}%"
    Creates a LaTeX formatted label for plots, highlighting one portion.
    Example: [0.2, 0.8] with bold_index=0 -> "$\\mathbf{20\\%}$/80%"
    """
    portions = [float(portions), 1 - float(portions)]
    if bold_index == 0:
        labels = rf"$\mathbf{{{math.ceil(portions[0]*100)}\%}}$/{math.ceil(portions[1]*100)}%"
    else:
        labels = rf"{math.ceil(portions[0]*100)}%/$\mathbf{{{math.ceil(portions[1]*100)}\%}}$"
    return labels

In [79]:
def plot_epsilon_delta_curves(sr, nm, cw, num_steps, budget_keys, portion_keys, colors, target_delta, target_epsilons):
    """
    Creates a plot showing the Epsilon vs. Delta privacy curve for each
    experimental portion, colored consistently.
    """
    fig = plt.figure(figsize=(6, 3))
    ax = plt.gca()
    
    # Store handles and labels as flat lists
    handles = []
    labels = []
    linewidth = 1.2

    for i, budget in enumerate(budget_keys):
        for p_idx, portion_name in enumerate(portion_keys):
            epsilons, deltas = _compute_epsilon_delta_curve(
                noise_multiplier=nm[i][p_idx]/cw[i][p_idx],
                deltas=np.logspace(-14, -1, 100),
                iterations=num_steps[i][p_idx],
                sampling_rate=sr[i][p_idx],
                clipping_norm=1,
            )
            label = generate_latex_label(portion_name, i)
            plot_handle, = ax.plot(epsilons, deltas, color=colors[i][p_idx + 3], alpha=0.9, linewidth=linewidth, label=label)
            
            # Append handles and labels in the desired order
            handles.append(plot_handle)
            labels.append(label)


    target_delta = target_delta
    target_epsilons = target_epsilons
    rest_handles = []
    rest_labels = []

    if target_delta:
        plot_handle = ax.axhline(target_delta, color='red', linestyle='--', linewidth=linewidth, label='Target $\delta$')
        rest_handles.append(plot_handle)
        rest_labels.append('Target $\delta$')
    
    if target_epsilons:
        for i, eps in enumerate(target_epsilons):
            plot_handle = ax.axvline(eps, color='red', linestyle=':', linewidth=linewidth, label='Target $\epsilon$' if i == 0 else None)
            if i == 0:
                rest_handles.append(plot_handle)
                rest_labels.append('Target $\epsilon$')
    
    # Correctly order and flatten the lists for the two-column legend
    handles_all = handles[:int(len(handles)/2)] + rest_handles + handles[int(len(handles)/2):]
    labels_all = labels[:int(len(handles)/2)] + rest_labels + labels[int(len(handles)/2):]
    
    # Add dummy entries to the right column to align the last items correctly
    for _ in rest_handles:
        handles_all.append(mpl.lines.Line2D([0], [0], color='none'))
        labels_all.append("")
    
    ax.set_yscale("log")
    ax.set_xlabel("Epsilon ($\epsilon$)")
    ax.set_ylabel("Delta ($\delta$, log scale)")
    
    # Pass the correctly ordered flat lists to the legend
    ax.legend(handles=handles_all, labels=labels_all, ncol=2, loc="upper right")
    plt.tight_layout()
    for ext in ["png", "svg"]: 
        fig.savefig(f"/vol/miltank/users/kaiserj/Clipping_vs_Sampling/extra_figs/epsilon_delta_curve_1.{ext}", dpi=300, bbox_inches='tight')
    plt.close(fig)





In [72]:
def _compute_epsilon_delta_curve(noise_multiplier, deltas, iterations, sampling_rate, clipping_norm):
    from dp_accounting.pld import privacy_loss_distribution
    """
    Compute epsilon for a range of deltas using the RDP accountant.
    """
    def compute_epsilon(noise_multiplier, delta, iterations, sampling_rate, clipping_norm):
        accountant = RDPAccountant()
        accountant.history = [(noise_multiplier/clipping_norm, sampling_rate, int(iterations))]
        return accountant.get_epsilon(delta)
    epsilons = []
    deltas2 = []
    for delta in deltas:
        epsilon = compute_epsilon(noise_multiplier, delta, iterations, sampling_rate, clipping_norm)
        epsilons.append(epsilon)
        deltas2.append(delta)
    return epsilons, deltas2

In [73]:
def _compute_tradeoff_envelope(params: Dict, alphas: np.ndarray) -> np.ndarray:
    from dp_accounting.pld import privacy_loss_distribution
    def profile2tradeoff(alpha, eps, delta):
        term1 = 1.0 - delta - np.exp(eps)*alpha
        term2 = (1.0 - delta - alpha)*np.exp(-eps)
        return np.maximum(0.0, np.maximum(term1, term2))

    def ADP2fDP(alphas, epsilons, deltas):
        betas = []
        for alpha in alphas:
            B = profile2tradeoff(alpha, np.asarray(epsilons), np.asarray(deltas))
            betas.append(np.max(B))
        return betas
    """Computes the trade-off curve for given params using PLD."""
    sigma = params["noise_multiplier"]
    p = params["sample_rate"]
    N = params["steps"]
    epsila = np.linspace(-10, 10, 10000)
    mech = privacy_loss_distribution.from_gaussian_mechanism(
        standard_deviation=sigma,
        sampling_prob=p).self_compose(N)
    deltas = mech.get_delta_for_epsilon(epsila)
    tradeoff = ADP2fDP(alphas, epsila, deltas)

    return alphas, tradeoff


In [74]:
def plot_privacy_tradeoff(nm, num_steps, sr, cw, budgets_keys, portion_keys, colors):
    """
    Creates a grid showing the Type I vs. Type II error trade-off and
    the resulting privacy advantage for each budget.
    """
    n_budgets = len(budget_keys)
    fig, axes = plt.subplots(1, 2, figsize=(6, 4 * 1), squeeze=False)
    alphas = np.linspace(0, 1, 10000)

    for r, budget_key in enumerate([budget_keys[0]]):
        ax_curve, ax_bar = axes[r, 0], axes[r, 1]
        advantages = {}
        
        for p_idx, portion_name in enumerate(portion_keys):
            params = {
                "noise_multiplier": float(nm[r][p_idx]) / float(cw[r][p_idx]),
                "steps": int(num_steps[r][p_idx]),
                "sample_rate": float(sr[r][p_idx]),
            }
            alphas, envelope = _compute_tradeoff_envelope(params, alphas)
            
            # Plotting
            label = generate_latex_label(portion_name, 1 if r == 1 else 0)
            # test = np.argmax(envelope, axis=0)
            # envelope = np.max(envelope, axis=0)
            print(f"r={r}, p_idx={p_idx}, {colors.shape}")
            ax_curve.plot(alphas, envelope, color=colors[r][p_idx + 3], linewidth=2, label=label)

            # from scipy.ndimage import gaussian_filter1d
            # smoothed_envelope = gaussian_filter1d(envelope, sigma=10)  # adjust sigma for more/less smoothing
            # ax_curve.plot(alpha_grid, smoothed_envelope, color="black", linewidth=0.2, label=label)

            # Advantage calculation
            trivial = 1 - alphas
            diff = trivial - envelope
            max_idx = np.argmax(np.nan_to_num(diff))
            advantages[p_idx] = diff[max_idx]
            ax_curve.plot([alphas[max_idx], alphas[max_idx]], [envelope[max_idx], trivial[max_idx]],
                            color=colors[r][p_idx + 3], linestyle=":", linewidth=1)

        # Format curve plot
        ax_curve.plot([0, 1], [1, 0], 'k:', label="_nolegend_")
        ax_curve.set_title(f"Budget ($\epsilon$) = {budget_key}")
        ax_curve.set_xlabel("Type I error $\\alpha$")
        ax_curve.set_ylabel("Type II error $\\beta$")
        ax_curve.grid(True)
        ax_curve.legend()

        # Format bar plot
        x_labels = [generate_latex_label(portion_name, 1 if r == 1 else 0) for portion_name in portion_keys]
        y_values = list(advantages.values())
        colors_barplot = colors[r][3:]
        ax_bar.bar(range(len(x_labels)), y_values, color=colors_barplot, width=0.5)
        ax_bar.set_ylabel("Advantage")
        ax_bar.set_xticks(range(len(x_labels)))
        ax_bar.set_xticklabels(x_labels, rotation=90)
        margin = 0.1 * (max(y_values) - min(y_values)) if len(y_values) > 1 and max(y_values) != min(y_values) else 0.1
        ax_bar.set_ylim(min(y_values) - margin, max(y_values) + margin)

    plt.tight_layout()
    for ext in ["png", "svg"]: 
        fig.savefig(f"/vol/miltank/users/kaiserj/Clipping_vs_Sampling/extra_figs/trade_off_curves_1.{ext}", dpi=300, bbox_inches='tight')
    plt.close(fig)


In [75]:
args = SimpleNamespace(
    accountant="rdp",  # Options: "rdp", "gdp", etc.
    individualize="sampling", # "sampling",  # Options: None, "clipping", "sampling"
    weights=None,  # Should be a list or None
    adapt_weights_to_budgets=True,  # Whether to adapt weights to budgets
    target_delta=1e-3,  # Default delta value for DP
    epochs=50,  # Number of training epochs
    max_grad_norm=1,  # Clipping norm for DP-SGD
    # noise_multiplier=1.0,  # Noise multiplier for DP
    n_data=10000,  # Number of data points
)


nm, sr, cw, num_steps = [], [], [], []
portions = [0.2, 0.4, 0.6, 0.8]
eps_1 = 3
epsilon_2 = 15
for portion in portions:
    dummy_train_loader = DataLoader(CustomDataset(num_samples=args.n_data), batch_size=128, shuffle=True)
    dummy_model = DummyResNet18()
    n_portion = int(portion * args.n_data)
    pp_budgets = [eps_1] * n_portion + [epsilon_2] * (args.n_data - n_portion)
    parameters = make_private(dummy_model, dummy_train_loader, pp_budgets, args)
    print(parameters)
    print()
    nm.append(parameters["noise_multiplier"])
    sr.append(parameters["sample_rate"])
    cw.append(parameters["max_grad_norms"])
    num_steps.append([args.epochs * len(dummy_train_loader), args.epochs * len(dummy_train_loader)])

{'budgets': [np.int64(3), np.int64(15)], 'max_grad_norms': [1, 1], 'sample_rate': [np.float64(0.0032958993688225746), np.float64(0.014976502396166325)], 'noise_multiplier': [0.6370068896484375, 0.6370068896484375]}

{'budgets': [np.int64(3), np.int64(15)], 'max_grad_norms': [1, 1], 'sample_rate': [np.float64(0.004333497025072575), np.float64(0.01815796084702015)], 'noise_multiplier': [0.688006640625, 0.688006640625]}

{'budgets': [np.int64(3), np.int64(15)], 'max_grad_norms': [1, 1], 'sample_rate': [np.float64(0.0058479318395257), np.float64(0.02288818545639515)], 'noise_multiplier': [0.7625062768554687, 0.7625062768554687]}

{'budgets': [np.int64(3), np.int64(15)], 'max_grad_norms': [1, 1], 'sample_rate': [np.float64(0.008193970657885075), np.float64(0.03057861514389515)], 'noise_multiplier': [0.8865056713867188, 0.8865056713867188]}



In [76]:
nm_t = torch.tensor(nm).t()
sr_t = torch.tensor(sr).t()
cw_t = torch.tensor(cw).t()
num_steps_t = torch.tensor(num_steps).t()
sr_t = torch.tensor(sr).t()
cw_t = torch.tensor(cw).t()
num_steps_t = torch.tensor(num_steps).t()
portion_keys = [str(por) for por in portions]
budget_keys = [eps_1, epsilon_2] 
target_delta = args.target_delta
target_epsilons = budget_keys
cmaps = [plt.get_cmap('Oranges'), plt.get_cmap('Blues')]
colors = [[cmap(x) for x in np.linspace(0, 1, len(portion_keys) + 3)] for cmap in cmaps]



In [80]:
plot_epsilon_delta_curves(sr_t, nm_t, cw_t, num_steps_t, budget_keys, portion_keys, colors, target_delta, target_epsilons)

In [78]:
plot_privacy_tradeoff(nm_t, num_steps_t, sr_t, cw_t, budget_keys, portion_keys, np.array(colors))

r=0, p_idx=0, (2, 7, 4)
r=0, p_idx=1, (2, 7, 4)
r=0, p_idx=2, (2, 7, 4)
r=0, p_idx=3, (2, 7, 4)
